<a href="https://colab.research.google.com/github/sarenales/VSR-Retrival/blob/main/ejemplo_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --single-branch --branch feature/add_transformers https://github.com/OFA-Sys/OFA


Cloning into 'OFA'...
remote: Enumerating objects: 5745, done.
remote: Counting objects: 100% (932/932), done.
remote: Compressing objects: 100% (256/256), done.
remote: Total 5745 (delta 710), reused 676 (delta 676), pack-reused 4813
Receiving objects: 100% (5745/5745), 97.78 MiB | 12.93 MiB/s, done.
Resolving deltas: 100% (2243/2243), done.
Updating files: 100% (3223/3223), done.


In [2]:
!pip install OFA/transformers/

Processing ./OFA/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3916748 sha256=cc03eb33a555ebb751a8bd287016edf41ca0833b3433525c5adde924313a918a
  Stored in directory: /tmp/pip-ephem-wheel-cache-9sni222z/wheels/0b/bc/ea/00b6b8998c20c4fe55affe6062a2cddda80308ef9bd5d5877c
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


# OFA infering


In [3]:
!git lfs install
!git clone https://huggingface.co/OFA-Sys/OFA-huge

Git LFS initialized.
Cloning into 'OFA-huge'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40 (from 1)
Unpacking objects: 100% (40/40), 539.60 KiB | 4.25 MiB/s, done.


In [4]:
from transformers.models.ofa.generate import sequence_generator

In [1]:
from transformers import OFATokenizer, OFAModel
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ofa_model = OFAModel.from_pretrained("./OFA-huge", use_cache=False).to(device)
ofa_tokenizer = OFATokenizer.from_pretrained("./OFA-huge")

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


./OFA-huge
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


In [2]:
from PIL import Image
from torchvision import transforms
mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 480

def load_image(image):
    patch_resize_transform = transforms.Compose([
            transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])
    patch_img = patch_resize_transform(image).unsqueeze(0).to(device)
    return patch_img

In [3]:
ejemplo = {'image': '000000574696.jpg',
  'image_link': 'http://images.cocodataset.org/train2017/000000574696.jpg',
  'caption+': 'The zebra is far away from the car.',
  'caption-': 'The zebra is close to the car.',
  'annotator_id': '2',
  'relation+': 'far away from',
  'relation-': 'close to',
  'split': 'train'}

In [169]:
ejemplo_nina = {'image': '000000293372.jpg',
  'image_link': 'http://images.cocodataset.org/train2017/000000293372.jpg',
  'caption+': 'The person is on top of the bed.',
  'caption-': 'The person is beneath the bed.',
  'annotator_id': '35',
  'relation+': 'on top of',
  'relation-': 'beneath',
  'split': 'train'}

In [120]:
ejemplo_gato = {'image': '000000209558.jpg',
  'image_link': 'http://images.cocodataset.org/train2017/000000209558.jpg',
  'caption+': 'The tv is beside the cat.',
  'caption-': 'The tv is far from the cat.',
  'annotator_id': '29',
  'relation+': 'beside',
  'relation-': 'far from',
  'split': 'train'}

In [106]:
ejemplo_caballo = {'image': '000000424912.jpg',
  'image_link': 'http://images.cocodataset.org/train2017/000000424912.jpg',
  'caption+': 'The person is on top of the horse.',
  'caption-': 'The person is beneath the horse.',
  'annotator_id': '27',
  'relation+': 'on top of',
  'relation-': 'beneath',
  'split': 'train'}

In [ ]:
ejemplo_zebra2 = { 'image': '000000556341.jpg',
  'image_link': 'http://images.cocodataset.org/train2017/000000556341.jpg',
  'caption+': 'The zebra is at the right side of the person.',
  'caption-': 'The zebra is at the left side of the person.',
  'annotator_id': '27',
  'relation+': 'at the right side of',
  'relation-': 'at the left side of',
  'split': 'train'}

#1

In [7]:
import numpy as np
import requests
import math
from torch import nn
from io import BytesIO
from tabulate import tabulate


In [8]:
def token_imag(url):
  response = requests.get(url)
  image = Image.open(BytesIO(response.content))
  patch_img = load_image(image.convert("RGB"))
  return patch_img, image

In [10]:
def token_cap(caption):
  cap = "Does the image describe the following sentence?  " + caption
  text = ofa_tokenizer([cap], padding=True, truncation=True, return_tensors="pt").to(device).input_ids
  return text,cap

In [11]:
def input_model(caption, patch_img):
  gen_output_0 = ofa_model.generate(caption, patch_images=patch_img,num_beams=1, no_repeat_ngram_size=1,  return_dict_in_generate=True , output_scores=True )
  ofa_caption_0 = ofa_tokenizer.batch_decode(gen_output_0[0], skip_special_tokens=True)[0].strip()
  return ofa_caption_0, gen_output_0

In [12]:
def get_probabilidades(gen_output_0):
  probabilities = nn.functional.softmax(gen_output_0[1][0], dim=-1)
  probYES = probabilities[0][tokens.get("yes")].item()
  probNO = probabilities[0][tokens.get("no")].item()
  return probYES, probNO

In [14]:
resultados = {}
tokens = ofa_tokenizer.get_vocab()

cont = 0

probYESP = 0.0
probNOP = 0.0
probYESN = 0.0
probNON = 0.0
respP = ""
respN = ""

url = ejemplo["image_link"]
patch_img,image = token_imag(url)

caption = token_cap(ejemplo["caption+"])[0]
ofa_caption_0, gen_output_0 = input_model(caption, patch_img)
respP = ofa_caption_0
probYESP, probNOP = get_probabilidades(gen_output_0)

caption = token_cap(ejemplo["caption-"])[0]
ofa_caption_1, gen_output_1 = input_model(caption, patch_img)
respN = ofa_caption_1
probYESN, probNON = get_probabilidades(gen_output_1)

ejemplo["PY+"] = probYESP
ejemplo["PN+"] = probNOP
ejemplo["response+"] = respP
ejemplo["PY-"] = probYESN
ejemplo["PN-"] = probNON
ejemplo["response-"] = respN


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [15]:

print(token_cap(ejemplo["caption+"])[1])
print("OFA caption + :",ofa_caption_0)
print("OFA caption - :",ofa_caption_1)

Does the image describe the following sentence?  The zebra is far away from the car.
OFA caption + : yes
OFA caption - : yes


#2

In [16]:
def acierta_1(PYP,PNP,PYN,PNN):
  DP = abs(PYP - PNP)
  DN = abs(PYN - PNN)

  if DN == 0.0:
    return 0
  if DP == 0.0:
    return 0

  if DP > DN :
    return 1
  return 0

In [20]:
datos_filtrados = []


rel_esp = ejemplo["relation+"]
acierto = acierta_1(ejemplo["PY+"],ejemplo["PN+"],ejemplo["PY-"],ejemplo["PN-"])
datos_filtrados.append({"relacion_espacial": rel_esp, "acierto": acierto})
if acierto == 1:
  print("ACIERTA")
else:
  print("FALLA ")

FALLA 


In [21]:
ejemplo["PY+"]

5.817415740239085e-07

In [22]:
ejemplo["PN+"]

9.238249987220115e-08

In [23]:
ejemplo["PY-"]

6.82908876115107e-07

In [24]:
ejemplo["PN-"]

1.764033896733963e-07

In [25]:
def acierta_2(PYP,PNP,PYN,PNN):
  if PYP == 0.0:
    return 0
  if PYN == 0.0:
    return 0

  total_prob_p = PYP + PNP
  normalized_prob_yes_p = PYP / total_prob_p
  normalized_prob_no_p = PNP / total_prob_p

  total_prob_n = PYN + PNN
  normalized_prob_yes_n = PYN / total_prob_n
  normalized_prob_no_n = PNN / total_prob_n

  if normalized_prob_yes_p > normalized_prob_no_n:
    return 1
  return 0


In [26]:
datos_filtrados = []


rel_esp = ejemplo["relation+"]
acierto = acierta_2(ejemplo["PY+"],ejemplo["PN+"],ejemplo["PY-"],ejemplo["PN-"])
datos_filtrados.append({"relacion_espacial": rel_esp, "acierto": acierto})
if acierto == 1:
  print("ACIERTA")
else:
  print("FALLA ")

ACIERTA


# 3

In [7]:
def token_imag(url):
  response = requests.get(url)
  image = Image.open(BytesIO(response.content))
  patch_img = load_image(image.convert("RGB"))
  return patch_img, image

In [8]:
def token_cap(caption):
  text = ofa_tokenizer([caption], padding=True, truncation=True, return_tensors="pt").to(device).input_ids
  return text

In [9]:
def probability_to_logit(p):
    p = p.item()
    if p == 0.0:
        return -10000
    return np.log(p)

In [10]:
def partes(caption):
  partes = caption.split()
  palabras = []
  for parte in partes:
    palabras.extend(parte.split("."))
  palabras[-1] = "."
  palabras.append("</s>")
  return palabras

In [11]:
import math
def pro_logs(logits_output, caption):
  sum_pro = 0
  mul_pro = 0
  suma_log1 = 0
  suma_log2 = 0
  datos = []
  datos.append(("Token", "Logit 1","Probabilidad de la siguiente", "Logit 2"))
  inputs_pal = []
  inputs_ids = []
  for i in partes(caption):
    inputs_pal.append(i)

  cont = 0
  for i in range(0, len(inputs_pal)-1):
    inputs_ids = (ofa_tokenizer(inputs_pal[i]).input_ids)
    for j in inputs_ids[1:-1]:
      probabilities = nn.functional.softmax(logits_output.logits, dim=-1)
      log_1 = logits_output.logits[0][cont][j].item()
      prob = probabilities[0][cont][j]
      log_2 = probability_to_logit(prob)
      datos.append((ofa_tokenizer.decode([j]), log_1, prob.item(), log_2))
      cont += 1
      suma_log1 += log_1
      sum_pro += prob
      mul_pro *= prob
      suma_log2 += log_2
  datos.append(("</s>", 0.0, 0.0, 0.0))
  print(tabulate(datos, headers="firstrow", tablefmt="grid"))
  print(f"Probabilidad media  : {(sum_pro/len(partes(caption))).item()} ")
  print(f"Mul probabilidades  : {mul_pro} ")
  print(f"Suma logits 1       : {suma_log1} ")
  print(f"Suma logits 2       : {suma_log2} ")
  return (sum_pro/len(partes(caption))).item() , mul_pro, suma_log1, suma_log2

In [17]:
import numpy as np
import requests
import math
from torch import nn
from io import BytesIO
from tabulate import tabulate

txt = "what does the image describe?"
inputs = token_cap(txt)

url = ejemplo["image_link"]
patch_img = token_imag(url)[0]

capP = token_cap(ejemplo["caption+"])
capN = token_cap(ejemplo["caption-"])



In [18]:
logits_output_P = ofa_model.forward(input_ids=inputs,patch_images=patch_img, decoder_input_ids=capP)

In [ ]:
logits_output_N = ofa_model.forward(input_ids=inputs,patch_images=patch_img, decoder_input_ids=capN)

In [ ]:
print("Caption +")
pro_medP , mul_proP, suma_log1P, suma_log2P = pro_logs(logits_output_P, captionP)


print("------------------------------------------------------------------------------")
print("Caption -")
pro_medN , mul_proN, suma_log1N, suma_log2N = pro_logs(logits_output_N, captionN)


In [ ]:
pro_med = 0
suma_log1 = 0
suma_log2 = 0
if pro_medP > pro_medN:
  pro_med = 1
if suma_log1P > suma_log1N:
  suma_log1 = 1
if suma_log2P > suma_log2N:
  suma_log2 = 1

print(f"Prob media :      {pro_med}")
print(f"Suma logit1:      {suma_log1}")
print(f"Suma logit2:      {suma_log2}")